Puzzle: https://code.golf/24-game

The 24 game is an arithmetical puzzle in which the objective is to find a way to combine four integers using only basic arithmetic operations (+, -, ×, ÷) to get a result of 24. Each integer must be used exactly once.

The variation we use is played with a standard 52-card deck, with integers ranging from 1 to 13. Print all solvable quadruples of integers. The integers of each quadruple should be printed in non-decreasing order.

Keep in mind that some solutions involve fractions. For example, the only solution to 1 3 4 6 is 6/(1-3/4).

I'm solving this by looping through all possible combinations of 4 cards from a deck of 52 cards.
- For each combination, create all possible permutations of the cards.
- For each permutation, combine the first two cards using all four operations, and check if the result is a valid solution.
- If it is, add the permutation to the list of solutions.

I'm keeping track of which permutations have already been checked to avoid duplicates.

I've also made this a general solution. That is, it works for any goal number and any number of cards.

In [160]:
from itertools import combinations, permutations
from time import perf_counter

class Game:
	epsilon = 1e-6
 
	def __init__(self, goal: int, cards: list[int], n_cards: int):
		self.goal = goal
		self.cards = cards
		self.n_cards = n_cards
  
		self.is_goal = set()
		self.not_goal = set()
		self.solution_time = 0
  
		self.solutions = self.solve()

	def __repr__(self):
		return f"Found {len(self.solutions)} solutions in {self.solution_time:.2f} seconds"

	def print_solutions(self):
		"""
  		print the output in the format specified by the puzzle description
		"""
		for s in self.solutions:
			print(" ".join([str(_) for _ in s]))
  
	@staticmethod
	def make_new_cards(cards: tuple[int, ...]):
		"""
		Given a length-k tuple of cards, return a generator of all possible length-k-1 tuples of cards
		resulting from applying the four operations to two cards.
  
		E.g. if cards = (1, 2, 3), then the output will be:
		- (1 + 2, 3) = (3, 3)
		- (1 - 2, 3) = (-1, 3)
		- (1 * 2, 3) = (2, 3)
		- (1 / 2, 3) = (0.5, 3)
		- (1 + 3, 2) = (4, 2)
		- (1 - 3, 2) = (-2, 2)
		- (1 * 3, 2) = (3, 2)
		- (1 / 3, 2) = (1/3, 2)
		- (1, 2 + 3) = (1, 5)
		- (1, 2 - 3) = (1, -1)
		- (1, 2 * 3) = (1, 6)
		- (1, 2 / 3) = (1, 2/3)
		"""
		for perm in set(permutations(cards)):
			yield (perm[0] + perm[1],) + perm[2:]
			yield (perm[0] - perm[1],) + perm[2:]
			yield (perm[0] * perm[1],) + perm[2:]
			if perm[1] != 0:
				yield (perm[0] / perm[1],) + perm[2:]

	def check_goal(self, cards: tuple[int, ...]) -> bool:
		"""
		Given a length-k tuple of cards, return True if the cards can be combined to form the goal number.
		If the tuple has only one card, check if it is within epsilon of the goal.
		If the cards cannot be combined to form the goal number, add the tuple to the not_goal set.
		Otherwise, add the tuple to the is_goal set and return True.
		"""
		if len(cards) == 1:
			return abs(cards[0] - self.goal) < self.epsilon

		sorted_cards = tuple(sorted(cards))
		if sorted_cards in self.not_goal:
			return False
		if sorted_cards in self.is_goal:
			return True
			
		for new_cards in self.make_new_cards(cards):
			if self.check_goal(new_cards):
				self.is_goal.add(sorted_cards)
				return True
		
		self.not_goal.add(sorted_cards)
		return False
						
	def solve(self):
		self.is_goal = set()
		self.not_goal = set()

		tic = perf_counter()
  
		for cards in combinations(self.cards, self.n_cards):
			self.check_goal(cards)
   
		self.solution_time = perf_counter() - tic
		return sorted(_tuple for _tuple in self.is_goal if len(_tuple) == self.n_cards)

In [161]:
game = Game(
	goal=24,
	cards=sorted(list(range(1, 14))*4),
	n_cards=4
)

print(game)

# game.print_solutions()

Found 1362 solutions in 0.22 seconds


In [153]:
# A more golfed version of the above code 
# It takes a little longer to run because it doesn't check for duplicates

import itertools as t
def f(h):
	if len(h)==1:
		return abs(h[0]-24)<1e-6
	for p in t.permutations(h):
		k=p[2:]
		if f((p[0]+p[1],)+k) or f((p[0]-p[1],)+k) or f((p[0]*p[1],)+k) or (p[1]!=0 and f((p[0]/p[1],)+k)):
			return 1
good_ones = []  # Remove this line
tic = perf_counter()  # Remove this line
for h in t.combinations_with_replacement(range(1,14),4):
	if f(h):
		good_ones.append(h)  # Remove this line
		# print(" ".join(str(_) for _ in h))  # Uncomment this line to print output
soln_time = perf_counter() - tic  # Remove this line
print(f"{len(good_ones)} solutions found in {soln_time:.2f} seconds")  # Remove this line

1362 solutions found in 1.84 seconds
